# Test ut funksjonaliteten i Python-Grew
Prøv ut funksjonene i [Python-grew-koden](https://gitlab.inria.fr/grew/python/-/tree/master/grew)

## Test ut innlasting av korpus

In [1]:
import grew
import tk

from pathlib import Path


In [2]:
grew.init()

[Grewpy] Port 8888 already used, failed to open socket
[Grewpy] Port 8889 already used, failed to open socket
connected to port: 8890


<Popen: returncode: None args: ['grewpy', '--caller', '35237', '--port', '88...>

In [3]:
#Load a corpus from a file of a string
#:param data: a file, a list of files or a CoNLL string representation of a corpus
corpus = grew.corpus('no_bokmaal-ud-train.conllu')

In [4]:
# :return: an integer index for latter reference to the corpus
corpus


1

In [5]:
grew.corpus_sent_ids(corpus)[:10]

['000001',
 '000002',
 '000003',
 '000004',
 '000005',
 '000006',
 '000007',
 '000008',
 '000009',
 '000010']

In [6]:
# Return the number of sentences in the corpus
grew.corpus_size(corpus)

15696

## Test ut søking i korpus

In [7]:
testpattern = '''pattern { GOV -[advcl]-> DEP }'''

In [8]:
# Search for [pattern] into [corpus_index]
corpus_results = grew.corpus_search(testpattern, corpus)

In [9]:
matches = corpus_results[:3]

In [10]:
# Count for [pattern] into [corpus_index]
corpus_count = grew.corpus_count(testpattern, corpus)
corpus_count

4197

In [11]:
# Graph for one sentence

# Search for [data] in previously loaded corpus
#    :param data: a sent_id (type string) or a position (type int)
#    :param corpus_index: an integer given by the [corpus] function
#    :return: a graph
mygraph = grew.corpus_get('015692', corpus)


In [12]:
# Search for [pattern] into [gr]
grew.search(testpattern, mygraph)

[]

In [13]:
# Søk etter flere setninger som matcher en regel i korpuset

grs_pattern = """ pattern {
    V [upos=VERB, Tense=Pres];
    V -[advcl]-> ADV;
    e: V -[nsubj]-> SUJ;
    P [upos=PROPN]; V -[obl]-> P;
    P -[nmod]-> A;
} """

print("matching nodes in the selected graph: ", grew.search(grs_pattern, mygraph))
print("# matching sentences in the corpus: ", grew.corpus_count(grs_pattern, corpus))
for sent in grew.corpus_search(grs_pattern, corpus):
    print(sent)

matching nodes in the selected graph:  []
# matching sentences in the corpus:  4
{'sent_id': '015690', 'matching': {'nodes': {'V': '6', 'SUJ': '7', 'P': '10', 'ADV': '3', 'A': '12'}, 'edges': {'e': {'source': '6', 'label': 'nsubj', 'target': '7'}}}}
{'sent_id': '011839', 'matching': {'nodes': {'V': '14', 'SUJ': '15', 'P': '25', 'ADV': '3', 'A': '20'}, 'edges': {'e': {'source': '14', 'label': 'nsubj', 'target': '15'}}}}
{'sent_id': '009104', 'matching': {'nodes': {'V': '10', 'SUJ': '11', 'P': '16', 'ADV': '5', 'A': '19'}, 'edges': {'e': {'source': '10', 'label': 'nsubj', 'target': '11'}}}}
{'sent_id': '009104', 'matching': {'nodes': {'V': '10', 'SUJ': '11', 'P': '16', 'ADV': '5', 'A': '14'}, 'edges': {'e': {'source': '10', 'label': 'nsubj', 'target': '11'}}}}


In [13]:
label = "advcl"
for sent in matches: 
    sent_id = sent.get('sent_id')
    graph = grew.corpus_get(sent_id, corpus)
    nodes = sent.get('matching').get('nodes')
    gov = nodes.get('GOV')
    dep = nodes.get('DEP')
    print(f"GOV: {graph[gov][0]['form']}, -[{label}]-> DEP: {graph[dep][0]['form']}")
    

GOV: innstilt, -[advcl]-> DEP: få
GOV: går, -[advcl]-> DEP: komme
GOV: må, -[advcl]-> DEP: Flyr


In [14]:
# Skriv grafen til en fil 
grew.save(mygraph, "graph_sample.json")

## Test ut graftransformasjoner

In [30]:
# GRS-testing 21.04.2022 (ILD)

In [15]:
grsfile = Path('teststrategy.grs')
grsfile.read_text()  # Sjekk innholdet i fila direkte

In [16]:
loaded_rule = grew.grs(grsfile.name)      # last inn med filnavn

In [22]:
grew.json_grs(loaded_rule)  # Sjekk innholdet av grs-en i json-format

{'filename': 'teststrategy.grs',
 'decls': [{'strat_name': 's_1', 'strat_def': {'Onf': {'Ref': 'verbrules'}}},
  {'package_name': 'verbrules',
   'decls': [{'rule_name': 'verbnonsense',
     'pattern': {'graph': [{'id': 'Ker 4',
        'node': {'node_name': 'V',
         'fs': [{'feature_name': 'Tense', '=': ['Pres']},
          {'feature_name': 'upos', '=': ['VERB']}],
         'next': [{'id': 'Ker 3',
           'label': {'edge_id': 'e', 'label_cst': {'pos': ['nsubj']}}},
          {'id': 'Ker 2',
           'label': {'edge_id': '__e_15__', 'label_cst': {'pos': ['obl']}}},
          {'id': 'Ker 1',
           'label': {'edge_id': '__e_14__',
            'label_cst': {'pos': ['advcl']}}}]}},
       {'id': 'Ker 3', 'node': {'node_name': 'SUJ', 'fs': [], 'next': []}},
       {'id': 'Ker 2',
        'node': {'node_name': 'P',
         'fs': [{'feature_name': 'upos', '=': ['PROPN']}],
         'next': [{'id': 'Ker 0',
           'label': {'edge_id': '__e_16__',
            'label_cst': {

In [19]:
#Apply rs or the last loaded one to [gr]
#:param grs_data: a graph rewriting system or a Grew string representation of a grs
#:param graph_data: the graph, either a str (in grew format) or a dict
#:param strat: the strategy (by default "main")
#:return: the list of rewritten graphs
grew.run(loaded_rule, mygraph, "s_1")

[{'0': [{'form': '__0__'}, [['root', '3']]],
  '1': [{'Gender': 'Neut',
    'Number': 'Sing',
    'PronType': 'Art',
    'form': 'Et',
    'lemma': 'en',
    'textform': 'Et',
    'upos': 'DET',
    'wordform': 'Et'},
   []],
  '2': [{'Definite': 'Ind',
    'Degree': 'Pos',
    'Gender': 'Neut',
    'Number': 'Sing',
    'form': 'helårs',
    'lemma': 'helårs',
    'textform': 'helårs',
    'upos': 'ADJ',
    'wordform': 'helårs'},
   []],
  '3': [{'Definite': 'Ind',
    'Gender': 'Neut',
    'Number': 'Sing',
    'form': 'reisemål',
    'lemma': 'reisemål',
    'textform': 'reisemål',
    'upos': 'NOUN',
    'wordform': 'reisemål'},
   [['det', '1'], ['amod', '2'], ['nmod', '5'], ['punct', '12']]],
  '4': [{'form': 'for',
    'lemma': 'for',
    'textform': 'for',
    'upos': 'ADP',
    'wordform': 'for'},
   []],
  '5': [{'Gender': 'Fem,Masc',
    'Number': 'Sing',
    'Person': '3',
    'PronType': 'Prs',
    'form': 'den',
    'lemma': 'den',
    'textform': 'den',
    'upos': 'PRO

In [21]:
grew.json_grs(loaded_rule)

{'filename': 'teststrategy.grs',
 'decls': [{'strat_name': 's_1', 'strat_def': {'Onf': {'Ref': 'verbrules'}}},
  {'package_name': 'verbrules',
   'decls': [{'rule_name': 'verbnonsense',
     'pattern': {'graph': [{'id': 'Ker 4',
        'node': {'node_name': 'V',
         'fs': [{'feature_name': 'Tense', '=': ['Pres']},
          {'feature_name': 'upos', '=': ['VERB']}],
         'next': [{'id': 'Ker 3',
           'label': {'edge_id': 'e', 'label_cst': {'pos': ['nsubj']}}},
          {'id': 'Ker 2',
           'label': {'edge_id': '__e_15__', 'label_cst': {'pos': ['obl']}}},
          {'id': 'Ker 1',
           'label': {'edge_id': '__e_14__',
            'label_cst': {'pos': ['advcl']}}}]}},
       {'id': 'Ker 3', 'node': {'node_name': 'SUJ', 'fs': [], 'next': []}},
       {'id': 'Ker 2',
        'node': {'node_name': 'P',
         'fs': [{'feature_name': 'upos', '=': ['PROPN']}],
         'next': [{'id': 'Ker 0',
           'label': {'edge_id': '__e_16__',
            'label_cst': {

In [35]:
tg = grew.graph(Path("input.grs").read_text())
tg

{'A': [{'label': 'A'}, [['L', 'B']]],
 'B': [{'label': 'B'}, [['L', 'C']]],
 'C': [{'label': 'C'}, []]}

In [37]:
# 
!grew transform -grs single.grs -config basic -strat p_1_onf -i input.gr -gr

100.00%                                                                                             
graph {
  N2 [label="A"] ;
  N1 [label="B"] ;
  N0 [label="C"] ;
  N2 -[L_12]-> N1;
  N1 -[L_12]-> N0;
}

